In [ ]:
import sys
sys.path.append('../lib_rag')
from config import get_config
from milvus_client import connect_milvus, delete_doc
from minio_client import get_minio_client

cfg = get_config()
mc = connect_milvus()
minio = get_minio_client()

# Estratégia simples: listar doc_ids em Milvus e checar se ainda existe objeto em MinIO.

def list_doc_ids(collection_name: str):
    results = mc.query(collection_name, filter='', output_fields=['doc_id'])
    ids = {r['doc_id'] for r in results}
    return ids

dense_ids = list_doc_ids(cfg.milvus_collection_dense)
print('Doc IDs em dense:', len(dense_ids))

# Aqui assumimos que source_bucket/source_path são iguais para dense/hybrid
to_delete = []
for doc_id in dense_ids:
    # pequena query para pegar source_bucket e source_path
    rows = mc.query(cfg.milvus_collection_dense, filter=f'doc_id == "{doc_id}"', output_fields=['source_bucket', 'source_path'])
    if not rows:
        continue
    sb = rows[0]['source_bucket']
    sp = rows[0]['source_path']
    try:
        minio.stat_object(sb, sp)
    except Exception:
        to_delete.append(doc_id)

print('Doc IDs a deletar (objeto não existe mais no MinIO):', len(to_delete))
for doc_id in to_delete:
    delete_doc(doc_id)
print('Sincronização de deleções concluída.')
